# Import Data From Brain Span 📨
When downloading any gene data from BrainSpan, you'll receive three distinct CSV files: Columns, Expression, and Rows. The provided code facilitates the creation of a new dataset focusing on the gene of interest.

Please note: The columns 'Gender' and 'Ethnicity' will be absent (copy and past from the 'UPS.xlsx' dataset available in this repository). For verification, you can refer to the 'Brain_Span_Documentation' file in the same repository.

In [1]:
import pandas as pd
def df_maker(file_col, file_row, file_expr):
    
    # Import the main dataframe
    df = pd.read_csv(file_col)

    # Import the file describing the rows/genes
    rows = pd.read_csv(file_row)

    # Import the metadata file
    # The first field of each row represents the gene ID to which the values (in log2) are associated!
    data = pd.read_csv(file_expr, header = None) # Indexing row and column names

    # Create a dictionary to associate the ID with the corresponding gene name
    id_to_gene = dict(zip(rows['id'], rows['gene-symbol']))
    
    # Set the value of the first field of each row (ID) as the row index
    data = data.set_index(0)

    # Create a new column (called gene_name) assigning the correct name to each row index
    data['gene_name'] = rows.set_index('id')['gene-symbol']
    data = data.reset_index()

    # Remove the ID column
    data = data.drop(columns=[0])

    # Move the 'gene_name' column to the first position
    cols = data.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data = data[cols]

    # Set 'gene_name' as the index
    data = data.set_index('gene_name')

    # Transpose the dataframe
    data_t = data.T

    # Rename the index name of the transposed column
    data_t = data_t.rename_axis(None, axis=1)

    # Reset the indices (otherwise, it would start from 1 instead of 0)
    data_t = data_t.reset_index(drop=True)
    
    # This line of code is not necessary! It only changes the name of some genes if you're interested
    #data_t.rename(columns={'C12orf51': 'HECTD4', 'KIAA0317': 'AREL1', 'UBE2CBP': 'UBE3D'}, inplace=True)
    
    # Reorder the columns in alphabetical order
    data_final = data_t.sort_index(axis=1)

    # Concatenate the initial df with data_final
    concat_data = pd.concat([df, data_final], axis=1)
    concat_data.drop(['donor_name', 'donor_color', 'structure_id',
                 'structure_name', 'structure_color','top_level_structure_id', 
                 'top_level_structure_name', 'top_level_structure_color'], axis = 1, inplace = True)
    age_groups = {
        'Early prenatal': ['8 pcw', '9 pcw', '12 pcw'],
        'Early mid-prenatal': ['13 pcw', '16 pcw', '17 pcw'],
        'Late mid-prenatal': ['19 pcw', '21 pcw', '24 pcw'],
        'Late prenatal': ['25 pcw', '26 pcw', '35 pcw', '37 pcw'],
        'Infancy': ['4 mos', '10 mos'],
        'Early childhood': ['1 yrs', '2 yrs', '3 yrs', '4 yrs'],
        'Late childhood': ['8 yrs', '11 yrs'],
        'Adolescence': ['13 yrs', '15 yrs', '18 yrs', '19 yrs'],
        'Adulthood': ['21 yrs', '23 yrs', '30 yrs', '36 yrs', '37 yrs', '40 yrs']
    }

    # Create a new column to represent the age group
    concat_data['age_group'] = concat_data['donor_age'].map({age: group for group, ages in age_groups.items() for age in ages})

    # Extract the age_group column
    age_group_column = concat_data['age_group']

    # Remove the age_group column from the DataFrame
    concat_data.drop(columns=['age_group'], inplace=True)

    # Insert the age_group column in the third position
    concat_data.insert(2, 'age_group', age_group_column)

    # Return the updated DataFrame
    return concat_data


In [2]:
col = 'Columns.csv'
rows = 'Rows.csv'
expr = 'Expression.csv'
df = df_maker(col,rows,expr)
pd.set_option('display.max_columns', None)
# save the df created
df.to_excel('<FileName>.xlsx', index = False)